In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
## bUILT-iN pACKAGES
import pickle
from collections import defaultdict
import json
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display

## lAST sUMMER
from romtoslp import *
from sentences import *
from DCS import *
import MatDB

## lAST yEAR
# from word_definite import *
# from nnet import *
# from heap_n_PrimMST import *
# from word_definite import *

In [3]:
loaded_SKT = pickle.load(open('../Simultaneous_CompatSKT_10K.p', 'rb'), encoding=u'utf-8')
loaded_DCS = pickle.load(open('../Simultaneous_DCS_10K.p', 'rb'), encoding=u'utf-8')

In [4]:
matDB = MatDB.MatDB()

In [10]:
from MatDB import *
import word_definite as WD
from heap_n_PrimMST import *
from nnet import *
"""
################################################################################################
######################  CREATE SEVERAL DATA STRUCTURES FROM SENTENCE/DCS  ######################
###########################  NODELIST, ADJACENCY LIST, GRAPH, HEAP #############################
################################################################################################
"""
def GetTrainingKit(sentenceObj, dcsObj):
    nodelist = GetNodes(sentenceObj)
    
    # Nodelist with only the correct_nodes
    nodelist2 = GetNodes(sentenceObj)
    nodelist2_to_correct_mapping = {}
    nodelist_correct = []
    search_key = 0
    first_key = 0
    for chunk_id in range(len(dcsObj.lemmas)):
        while nodelist2[first_key].chunk_id != chunk_id:
            first_key += 1
        for j in range(len(dcsObj.lemmas[chunk_id])):
            search_key = first_key
            while (nodelist2[search_key].lemma != rom_slp(dcsObj.lemmas[chunk_id][j])) or (nodelist2[search_key].cng != dcsObj.cng[chunk_id][j]):
                search_key += 1
                if search_key >= len(nodelist2) or nodelist2[search_key].chunk_id > chunk_id:
                    break
    #         print((rom_slp(dcsObj.lemmas[chunk_id][j]), dcsObj.cng[chunk_id][j]))
    #         print(nodelist[search_key])
            nodelist2_to_correct_mapping[len(nodelist_correct)] = search_key
            nodelist_correct.append(nodelist2[search_key])
    return (nodelist, nodelist_correct, nodelist2_to_correct_mapping)
    

def GetGraph(nodelist, neuralnet):
    conflicts_Dict = Get_Conflicts(nodelist)

    featVMat = Get_Feat_Vec_Matrix(nodelist, conflicts_Dict)

    (WScalarMat, SigmoidGateOutput) = Get_W_Scalar_Matrix_from_FeatVect_Matrix(featVMat, nodelist, conflicts_Dict, neuralnet)
    return (conflicts_Dict, featVMat, WScalarMat, SigmoidGateOutput)

In [396]:
"""
################################################################################################
##############################  MAIN FUNCTION  #################################################
################################################################################################
"""
def main(loaded_SKT, loaded_DCS):
    # Train
    totalBatchToTrain = 1000
    filePerBatch = 30
    iterationPerBatch = 5
    _skt_file_list = list(loaded_SKT.keys())
    for iterout in range(totalBatchToTrain):
        # Change current batch
        print('ITERATION OUT', iterout)
        perm = np.random.permutation(len(loaded_SKT))[0:filePerBatch]
        print('Permutation: ', perm)
        # trainer.Load('outputs/neuralnet_trained.p')
        
        # Run few times on same set of files
        for iterin in range(iterationPerBatch):
            print('ITERATION IN', iterin)        
            for fn in perm:
                sentenceObj = loaded_SKT[_skt_file_list[fn]]
                dcsObj = loaded_DCS[_skt_file_list[fn]]
                # trainer.Save('outputs/saved_trainer.p')
                trainer.Train(sentenceObj, dcsObj)
                
def test(loaded_SKT, loaded_DCS, perm = None, n_testSet = 10):
    _skt_file_list = list(loaded_SKT.keys())
    total_lemma = 0;
    correct_lemma = 0;

    total_word = 0;
    correct_word = 0;
    file_counter = 0
    if perm is None:
        n_testSet = min(n_testSet, len(_skt_file_list))
        perm = np.random.permutation(len(_skt_file_list))[0:n_testSet]
        print('Permutation: ', perm)
        for fn in perm:
            file_counter += 1
            sentenceObj = loaded_SKT[_skt_file_list[fn]]
            dcsObj = loaded_DCS[_skt_file_list[fn]]
            print(file_counter, end='. ')
            try:
                (word_match, lemma_match, n_dcsWords) = trainer.Test(sentenceObj, dcsObj)
                total_word += n_dcsWords
                correct_lemma += lemma_match
                correct_word += word_match
            except:
                print('Failed')
    else:
        print('Permutation: ', perm)
        for fn in perm:
            file_counter += 1
            sentenceObj = loaded_SKT[_skt_file_list[fn]]
            dcsObj = loaded_DCS[_skt_file_list[fn]]
            print(file_counter, end='. ')
            try:
                (word_match, lemma_match, n_dcsWords) = trainer.Test(sentenceObj, dcsObj)
                total_lemma += n_dcsWords
                total_word += n_dcsWords
                correct_lemma += lemma_match
            except:
                print('Failed!')
            
            correct_word += word_match

    print('Micro Accuracy[lemma]: {}'.format(100*correct_lemma/total_word))
    print('Micro Accuracy[word]: {}'.format(100*correct_word/total_word))
    

In [324]:
# NEW FUNCTION
def GetLoss(_mst_adj_graph, _mask_de_correct_edges, _negLogLikelies):
    _negLogLikelies = _negLogLikelies.copy()
    _negLogLikelies[~_mst_adj_graph] = 0
    _negLogLikelies[~_mask_de_correct_edges] *= -1 # BAKA!!! Check before you try to fix this again
    return np.sum(_negLogLikelies)    

In [388]:
class Trainer:
    def __init__(self):
        self._edge_vector_dim = WD._edge_vector_dim
        self._full_cnglist = list(WD.mat_cngCount_1D)
        self.neuralnet = NN(self._edge_vector_dim, 200)
        self.history = defaultdict(lambda: list())
        
    def Reset(self):
        self.neuralnet = NN(self._edge_vector_dim, 200)
        self.history = defaultdict(lambda: list())
        
    def Save(self, filename):
        pickle.dump({'nnet': self.neuralnet, 'history': dict(self.history)}, open(filename, 'wb'))
        
    
    def Load(self, filename):
        o = pickle.load(open(filename, 'rb'))
        self.neuralnet = o['nnet']
        self.history = defaultdict(lambda: list(), o['history'])    
        
    def Test(self, sentenceObj, dcsObj):
        neuralnet = self.neuralnet
        minScore = np.inf
        minMst = None
        try:
            (nodelist, nodelist_correct, _) = GetTrainingKit(sentenceObj, dcsObj)
            # nodelist = GetNodes(sentenceObj)
        except IndexError:
            print('\x1b[31mError with {} \x1b[0m'.format(sentenceObj.sent_id))
            return (0, 0, 0)
            
        conflicts_Dict = Get_Conflicts(nodelist)
        conflicts_Dict_correct = Get_Conflicts(nodelist_correct)
        
        featVMat = Get_Feat_Vec_Matrix(nodelist, conflicts_Dict)
        featVMat_correct = Get_Feat_Vec_Matrix(nodelist_correct, conflicts_Dict_correct)
        
        (WScalarMat, SigmoidGateOutput) = Get_W_Scalar_Matrix_from_FeatVect_Matrix(featVMat, nodelist, conflicts_Dict, neuralnet)
        # print(WScalarMat)
        # Get all MST
        for source in range(len(nodelist)):
            (mst_nodes, mst_adj_graph) = MST(nodelist, WScalarMat, conflicts_Dict, source)
            print('.', end = '')
            score = GetMSTWeight(mst_nodes, WScalarMat)
            if(score < minScore):
                print('.', end = '')
                minScore = score
                minMst = mst_nodes
        dcsLemmas = [[rom_slp(l) for l in arr]for arr in dcsObj.lemmas]
        word_match = 0
        lemma_match = 0
        for chunk_id, wdSplit in mst_nodes.items():
            for wd in wdSplit:
                # Match lemma
                search_result = [i for i, j in enumerate(dcsLemmas[chunk_id]) if j == wd.lemma]
                if len(search_result) > 0:
                    lemma_match += 1
                # Match CNG
                for i in search_result:
                    if(dcsObj.cng[chunk_id][i] == str(wd.cng)):
                        word_match += 1
                        # print(wd.lemma, wd.cng)
                        break
        dcsLemmas = [l for arr in dcsObj.lemmas for l in arr]
        print('\nFull Match: {}, Partial Match: {}, OutOf {}, NodeCount: {}, '.\
              format(word_match, lemma_match, len(dcsLemmas), len(nodelist)))
        return (word_match, lemma_match, len(dcsLemmas))
    
    def Train(self, sentenceObj, dcsObj):
        try:
            (nodelist, nodelist_correct, nodelist_to_correct_mapping) = GetTrainingKit(sentenceObj, dcsObj)
        except IndexError as e:
            # print('\x1b[31mError with {} \x1b[0m'.format(sentenceObj.sent_id))
            # print(e)
            return
        
        """ CREATE A MASK FOR ALL EDGES BETWEEN CORRECT NODE PAIRS"""
        mask_de_correct_edges = np.ndarray((len(nodelist), len(nodelist)), np.bool)*False
        for n1 in nodelist_to_correct_mapping.values():
            for n2 in nodelist_to_correct_mapping.values():
                if n1 != n2:
                    mask_de_correct_edges[n1, n2] = 1
                    mask_de_correct_edges[n2, n1] = 1
                
        
        """ FOR MST OF GRAPH WITH ONLY CORRECT SET OF NODES """
        (conflicts_Dict_correct, featVMat_correct, WScalarMat_correct, _) = GetGraph(nodelist_correct, self.neuralnet)
        source = 0
        (mst_nodes_correct, mst_adj_graph_correct_0) = MST(nodelist_correct, WScalarMat_correct, conflicts_Dict_correct, source)
        W_star = GetMSTWeight(mst_nodes_correct, WScalarMat_correct) # Total weight of correct MST
        
        """ FOR ALL POSSIBLE MST FROM THE COMPLETE GRAPH """
        (conflicts_Dict, featVMat, WScalarMat, SigmoidGateOutput) = GetGraph(nodelist, self.neuralnet)
        Total_Loss = 0

        # Convert correct spanning tree graph adj matrix to actual marix dimensions
        # Create full-size adjacency matrix for correct_mst
        nodelen = len(nodelist)
        mst_adj_graph_correct = np.ndarray((nodelen, nodelen), np.bool)*False
        for i in range(mst_adj_graph_correct_0.shape[0]):
            for j in range(mst_adj_graph_correct_0.shape[1]):
                mst_adj_graph_correct[nodelist_to_correct_mapping[i], nodelist_to_correct_mapping[j]] = \
                mst_adj_graph_correct_0[i, j]
#         for p in mask_de_correct_edges*1:
#             print()
#             for q in p:
#                 print(q, end = ' ')

        """ For each node - Find MST with that source"""
        dLdOut = np.zeros(WScalarMat.shape)
        for source in range(len(nodelist)):
            (mst_nodes, mst_adj_graph) = MST(nodelist, WScalarMat, conflicts_Dict, source)
            print('.', end = '')

            """ Gradient Descent"""
            # Total_Loss += (W_star - GetMSTWeight(mst_nodes, WScalarMat))
            Total_Loss += GetLoss(mst_adj_graph, mask_de_correct_edges, WScalarMat)
            # dLdOut_inner = 1 / WScalarMat
            
            # For new loss function
            dLdOut_inner = (1 - SigmoidGateOutput)
            dLdOut_inner[~mst_adj_graph] = 0 # Edges not in the current tree
            dLdOut_inner[mst_adj_graph^mask_de_correct_edges] = -dLdOut_inner[mst_adj_graph^mask_de_correct_edges]
            
            dLdOut += dLdOut_inner
#         print('dLdOut: ')
#         print(dLdOut)
        self.neuralnet.Back_Prop(dLdOut/len(nodelist), len(nodelist), featVMat)        
        Total_Loss /= len(nodelist)
        self.history[sentenceObj.sent_id].append(Total_Loss)
        print("\nFileKey: %s, Loss: %6.3f, Original MSTScore: %6.3f" % (sentenceObj.sent_id, Total_Loss, W_star))

In [389]:
# trainer.neuralnet.U

In [394]:
trainer = None
def InitModule(_matDB):
    global WD, trainer
    WD.word_definite_extInit(_matDB)
    trainer = Trainer()
InitModule(matDB)

443


In [375]:
"""
################################################################################################
################################################################################################
################################################################################################
"""
# if __name__ == '__main__':
# loaded_SKT = pickle.load(open('../Simultaneous_CompatSKT_10K.p', 'rb'))
# loaded_DCS = pickle.load(open('../Simultaneous_DCS_10K.p', 'rb'))
# main(loaded_SKT, loaded_DCS)
trainer.Train(loaded_SKT['170095.p2'], loaded_DCS['170095.p2'])
# print ("Not Implemented")

................................
FileKey: 170095, Loss: -0.014, Original MSTScore:  0.000


In [382]:
main(loaded_SKT, loaded_DCS)

ITERATION OUT 0
Permutation:  [9049 4123 9209 3363 3732 9491 4685 6540 6783 8466 5499   68 1354 8741 7970
 3033 8972 7686 9949 2784 7792 6012 6949 7923 1410 4018 1368 6655 3550 4390]
ITERATION IN 0
..................................
FileKey: 390330, Loss:  0.004, Original MSTScore:  0.109
......................
FileKey: 404755, Loss:   -inf, Original MSTScore:  0.013
......................
FileKey: 27269, Loss: -0.682, Original MSTScore:  0.000
.................................................
FileKey: 2814, Loss: -3.490, Original MSTScore:  2.965
...
FileKey: 354131, Loss: -0.124, Original MSTScore:  0.001
........................
FileKey: 405066, Loss: -0.031, Original MSTScore:  0.000
.......................................
FileKey: 164971, Loss: -0.000, Original MSTScore:  0.000
..............................................
FileKey: 43104, Loss:   -inf, Original MSTScore:  1.000
...............................................................................
FileKey: 208479, Loss: 

d:\program files\anaconda3\lib\site-packages\ipykernel\__main__.py:115: RuntimeWarning: invalid value encountered in double_scalars


..........................................................
FileKey: 11773, Loss:   -inf, Original MSTScore:  3.010
....................................
FileKey: 308691, Loss: -1.918, Original MSTScore:  1.000
........................................................
FileKey: 32929, Loss: -0.500, Original MSTScore:  0.000
..........................................
FileKey: 279247, Loss: -1.500, Original MSTScore:  0.000
.......................
FileKey: 39839, Loss:   -inf, Original MSTScore:  0.000
...................................................
FileKey: 259409, Loss:   -inf, Original MSTScore:  0.000
...........
FileKey: 275162, Loss: -1.382, Original MSTScore:  0.017
.............................
FileKey: 296666, Loss:   -inf, Original MSTScore:  0.000
..........................................................................
FileKey: 162640, Loss: -0.838, Original MSTScore:  0.000
............................
FileKey: 117937, Loss:   -inf, Original MSTScore:  0.020
...............

KeyboardInterrupt: 

In [397]:
test(loaded_SKT, loaded_DCS, n_testSet=1000)
# test(loaded_SKT, loaded_DCS, perm = [3486, 8594, 21, 1862, 7800, 9822, 5341, 2432, 1144, 7991, 4968, 8473, 7926, 5625, 8442, 4822, 8762, 3555, 6457, 5801, 5358, 1171, 3438, 5789, 6047, 459, 2340, 4969, 8032, 570, 1580, 7878, 3864, 8989, 36, 174, 2056, 6183, 9514, 375, 3496, 9439, 4934, 7531, 505, 8299, 4341, 4541, 9462, 7674, 4466, 6519, 760, 3654, 8972, 4362, 9971, 8726, 9089, 2139, 8322, 647, 9990, 2458, 4765, 8820, 1354, 5539, 4698, 4836, 8866, 5744, 3879, 1224, 7981, 4692, 7407, 8597, 3709, 4732, 662, 7870, 6169, 217, 8270, 6888, 2370, 684, 8449, 2099, 7681, 2310, 2511, 8878, 8756, 514, 4605, 4565, 2455, 5592])

Permutation:  [ 699  170  303 4179  214 8729 7456 5253 6147  972 3823 3660 6114 2770 3631
 3797 8122 6773 6438 4543  895 8837 9485  183  344 7858 5567 9225 7670  426
 5954 9994 7975 8821 4212 6298 2228 4404 4300 4334 4270 9087 3480 8895 7732
 1263 4635 1696 8938 2441 8128 9297 8962 4086 8183 5638 8521 2308 8257 6348
 9371  788 1517 3964 6277 3930 2190 9269 3859 1617 8495 3869 8958 8957 8896
 2755 3649 3325 4117 3855 3731 6338  249 6021 3284 8047 4720  178 8203 1694
 4869  978 1615 2589 4294  636 9515 4128 7822 1274 9180 6424  468 1786 1678
 2902 7687 2122 2766  747 3169 3744 3419 1257 1612   69 6601 8113 1761  698
 2468 2541 5392 2079 7498 4174 3458 5603 7385 7671 5222 6972  446 3293 4218
 9367 5214  312 9150 3316 2012 6236 7842  718 5970 2964 1002 1235 6311 6955
 6526 3598 4359 8614 1230 5649 5069 9300 9329  731 3830 8645 9529 6246 2413
 2130 8725 5696 1944 4795 6980 2189 9870 3938 1991 8683 5323  880 6931 1541
 2099 2773 6454 8794 2544 1001 2365 5131 2381 4323 8728 1354 6541 4716 175

In [381]:
test(loaded_SKT, loaded_DCS, n_testSet=1000)

Permutation:  [5711 6997 3551 4677 1369 8029  358 3074 5829 7030  208 5969 4812 3254 3949
 3561 3573 5197 2153  408 4209 3626  643 8186 9588 7600 6660 1022  200 8968
 7097  754 9289 1893 9748  576 1065 8178 8715 3327 5260  173 8122 4792 4582
 4564 5566 5778 6863 6996 7495 7865 5660 5648 4182 4100 5624 3076 5480 8486
  699 6917 4628 5850 9812  389 4181 2035 2714 9724 3722 5571 4371 7623 3343
  279  819  947 7470 6465  652 3265 1077 4700 9644 1748  236 3812 2098 7855
 5843 1062 9711 9316 3373 2608 1737 9509  594 2891 9673  737 6551 9557 9309
 2692  334 1356 5228 4495 5399 7588 2006 8261  776 9167  959 9484 3630 3037
 6981 6370 4920 1136 5578 4290 2257 2542 1063 1880 9197 1445 4579 1182 9648
  532 1483  437 3045  720  985 7370  342  657 1089 4642 6369  528 8834 7528
 8474 9589 8328 4063  274 1046 1498 7944 9577 8643 5478  416 7975 3129 8082
 3408  984 3761 8932 9796  619 1599 2018 4681 5117 5393 9849 7679 5812  862
 6051 8775 7747 2671 6731 8088 9842 6264 4133 5890 4372 6679 4817 4918 164

In [384]:
trainer.neuralnet.W

array([[  373.1818749 ,    93.33429557,   995.79257945, ...,
          -96.91720146,    81.01281295,    29.26593169],
       [  338.44253789,    35.42980376,  1823.96108288, ...,
           26.00471189,    74.12211956,   -91.31908744],
       [    3.614079  ,    37.42117537,   -80.76814807, ...,
          -49.90456286,   -95.353375  ,    64.45850842],
       ..., 
       [ 1538.04104715,    31.84052596,  3447.16299766, ...,
          -12.02931747,    92.93596019,   -71.44181617],
       [  308.90965639,    99.65336422,  4667.1902688 , ...,
           87.39209397,    17.22356181,   -23.08250093],
       [    0.31657415,    44.30685523,  -342.68512523, ...,
          -53.97662606,   -36.6685354 ,    13.27014288]])

In [380]:
trainer.neuralnet.W

array([[  346.84899977,    87.62658988,   953.9122149 , ...,
          -96.91720146,    81.01281295,    29.09381347],
       [  275.94200333,    21.88526968,  1724.94630827, ...,
           26.00471189,    74.12211956,   -91.73162305],
       [    6.55110571,    37.81797125,   -80.67550415, ...,
          -49.90456286,   -95.353375  ,    64.46963047],
       ..., 
       [ 1406.5979983 ,     3.34664157,  3238.86073541, ...,
          -12.02931747,    92.93596019,   -72.3090234 ],
       [  193.85301111,    74.72228716,  4484.99891372, ...,
           87.39209397,    17.22356181,   -23.84230679],
       [    0.3165742 ,    44.30685523,  -342.68512519, ...,
          -53.97662606,   -36.6685354 ,    13.27080634]])